# Constant Phase Shift of Periodic Signals

Constant phase shift of a periodic signal is performed in the DFT domain,
which is equivalent to the periodic convolution of the constant phase shifter.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import util
from util import db
from scipy.signal import resample, kaiser, freqz
from os.path import join

## Periodic Repetition

Periodic repetition of constant phase shifters for period $M$ reads

\begin{align}
\tilde{h}[n] = \sum_{k=-\infty}^{\infty} h[n - k M]
\end{align}

where $h[n]$ is the impulse response of a constant phase shifter

\begin{align}
h[n] =
\begin{cases}
\cos\varphi,& n=0\\
0,& \text{$n\neq 0$ even}\\
\frac{-2}{\pi n}\sin\varphi,& \text{$n$ odd}.
\end{cases}
\end{align}

The latter is a linear combination of two components
that form an analytic signal pair,

\begin{align}
h[n] = \cos\varphi \cdot \delta[n] - \sin\varphi \cdot h_\text{H}[n]
\end{align}

where $\delta[n]$ denotes the unit impulse function
and $h_\text{H}[n]$ its Hilbert transformer,

\begin{align}
h_\text{H}[n] =
\begin{cases}
0,& \text{$n$ even}\\
\frac{2}{\pi n},& \text{$n$ odd}.
\end{cases}
\end{align}

The infinite summation can be also decomposed as

\begin{align}
\tilde{h}[n] =
\cos\varphi \sum_{k=-\infty}^{\infty} \delta[n - kM]
- \sin\varphi \sum_{k=-\infty}^{\infty} h_\text{H}[n - kM].
\end{align}

Since $\tilde{h}[n]$ is periodic, only one period needs to be specified,
e.g. $\tilde{h}[n], n=0,\ldots,M-1$.

In [ ]:
def periodic_constant_phase_shifter_summation(period, phase_angle, order=10):
    h = np.zeros(period)
    for m in range(-order, order + 1):
        idx = m * period + np.arange(period)
        h += util.discrete_ir_constant_phase(idx, phase_angle)
    return h

phase_angle = 0.25 * np.pi
Orders = 0, 1, 20

N_even, N_odd = 32, 31

fig, Ax = plt.subplots(figsize=(15, 3), ncols=3, sharey=True)
for order, ax in zip(Orders, Ax):
    h_sum_even = periodic_constant_phase_shifter_summation(N_even, phase_angle, order)
    ax.stem(h_sum_even)
    ax.grid()
    ax.set_xlabel('$n$ / samples')
    ax.set_title('$N={:0.0f}$ / {:0.0f} Term(s)'.format(N_even, 2 * order + 1))

fig, Ax = plt.subplots(figsize=(15, 3), ncols=3, sharey=True)
for order, ax in zip(Orders, Ax):
    h_sum_odd = periodic_constant_phase_shifter_summation(N_odd, phase_angle, order)
    ax.stem(h_sum_odd)
    ax.grid()
    ax.set_xlabel('$n$ / samples')
    ax.set_title('$N={:0.0f}$ / {:0.0f} Term(s)'.format(N_odd, 2 * order + 1))    

### Even $M$

$n=0$

\begin{align}
\tilde{h}[0]
&= \cos\varphi - \sin\varphi \sum_{k=-\infty}^{\infty} h_\text{H}[-kM]\\
&= \cos\varphi
\end{align}

The second summation vanishes since $h_\text{H}[- k M] = 0, \forall k\in\mathbb{Z}$.

$n \neq 0$ even

\begin{align}
\tilde{h}[n]
&= \cos\varphi \sum_{k=-\infty}^{\infty} \delta[n - kM]
- \sin\varphi \sum_{k=-\infty}^{\infty} h_\text{H}[n - kM]\\
&= 0
\end{align}

Again, the second summation vanishes because $n - kM$
is always an even number and thus $h_\text{H}[n - kM] = 0, \forall k\in\mathbb{Z}$.

$n$ odd

\begin{align}
\tilde{h}[n]
&= \cos\varphi \sum_{k=-\infty}^{\infty} \delta[n - kM]
- \sin\varphi \sum_{k=-\infty}^{\infty} h_\text{H}[n - kM]\\
&= 0 + \frac{-2 \sin\varphi}{\pi} \left[
\frac{1}{n} + \sum_{k'=1}^{\infty}
\left(\frac{1}{n - k'M} + \frac{1}{n + k'M}\right) \right]\\
&= \frac{-2 \sin\varphi}{\pi} \left[
\frac{1}{n} + 2n \sum_{k'=1}^{\infty}
\left(\frac{1}{n^2 + (k'M)^2}\right) \right]\\
&= \frac{-2 \sin\varphi}{\pi M} \left[
\frac{M}{n} + \frac{2n}{M} \sum_{k'=1}^{\infty}
\left(\frac{1}{(n/M)^2 - k'^2}\right) \right]\\
&= \frac{-2 \sin\varphi}{M}
\cot\left(\frac{\pi n}{M}\right)
\end{align}

In the last equality, the series expansion of the cotangent function is exploited,

\begin{align}
\pi \cot(\pi z) = \frac{1}{z} + 2 z \sum_{k=1}^{\infty} \frac{1}{z^2 - k^2}.
\end{align}

To summarize,

\begin{align}
\tilde{h}[n] =
\begin{cases}
\cos\varphi,& n=0\\
0,& \text{$n\neq 0$ even}\\
\frac{-2\sin\varphi}{M} \cot\left(\tfrac{\pi n}{M}\right),
& \text{$n$ odd}
\end{cases}
\end{align}

for even $M$.

### Odd $M$

$n=0$

\begin{align}
\tilde{h}[0]
&= \cos\varphi - \sin\varphi \sum_{k=-\infty}^{\infty} h_\text{H}[-kM]\\
&= \cos\varphi
\end{align}

In the summation, $h_\text{H}[-kM] = 0$ for even $k$,
and $h_\text{H}[-kM] + h_\text{H}[kM] = 0$ for odd $k$.

$n \neq 0$ even

\begin{align}
\tilde{h}[n]
&= \cos\varphi \sum_{k=-\infty}^{\infty} \delta[n - kM]
- \sin\varphi \sum_{k=-\infty}^{\infty} h_\text{H}[n - kM]\\
&= 0 + -\sin\varphi
\left[h_\text{H}[n + M] +  \sum_{k'=1}^{\infty} h_\text{H}[n + M + 2k' M]
+ h_\text{H}[n + M - 2k'M]\right]\\
&= \frac{-2\sin\varphi}{\pi} \left[
\frac{1}{n'} + \sum_{k'=1}^{\infty}
\left(\frac{1}{n' + 2k'M} + \frac{1}{n' - 2k'M}\right)
\right]\\
&= \frac{-2\sin\varphi}{\pi} \left[
\frac{1}{n'} + 2n \sum_{k'=1}^{\infty}
\left(\frac{1}{n'^2 - (2k'M)^2}\right)
\right]\\
&= \frac{-\sin\varphi}{\pi M} \left[
\frac{2M}{n'} + \frac{n}{M} \sum_{k'=1}^{\infty}
\left(\frac{1}{n'^2 - (2k'M)^2}\right)
\right]\\
&= \frac{-\sin\varphi}{M}
\cot\left(\frac{\pi (n + M)}{2M}\right)
\end{align}

In the third equality, the variable is substituted by $n' = n + M$.
The series expansion of the cotangent function is used in the last equality.

$n \neq 0$ odd

\begin{align}
\tilde{h}[n]
&= \cos\varphi \sum_{k=-\infty}^{\infty} \delta[n - kM]
- \sin\varphi \sum_{k=-\infty}^{\infty} h_\text{H}[n - kM]\\
&= 0 + -2\sin\varphi
\left[h_\text{H}[n] + \sum_{k'=1}^{\infty} h_\text{H}[n + 2k' M]
+ h_\text{H}[n - 2k'M]\right]\\
&= \frac{-2\sin\varphi}{\pi} \left[
\frac{1}{n} + \sum_{k'=1}^{\infty}
\left(\frac{1}{n + 2k'M} + \frac{1}{n - 2k'M}\right)
\right]\\
&= \frac{-2\sin\varphi}{\pi} \left[
\frac{1}{n} + 2n \sum_{k'=1}^{\infty}
\left(\frac{1}{n^2 - (2k'M)^2}\right)
\right]\\
&= \frac{-\sin\varphi}{\pi M} \left[
\frac{2M}{n} + \frac{n}{M} \sum_{k'=1}^{\infty}
\left(\frac{1}{n^2 - (2k'M)^2}\right)
\right]\\
&= \frac{-\sin\varphi}{M}
\cot\left(\frac{\pi n}{2M}\right)
\end{align}

Again, the series expansion of the cotangent function is used in the last equality.

To summarize,

\begin{align}
\tilde{h}[n] =
\begin{cases}
\cos\varphi,& n=0\\
\frac{-\sin\varphi}{M} \cot\left(\tfrac{\pi (n + M)}{2M}\right),& \text{$n\neq 0$ even}\\
\frac{-\sin\varphi}{M} \cot\left(\tfrac{\pi n}{2M}\right),
& \text{$n$ odd}
\end{cases}
\end{align}

for odd $M$.

In [ ]:
def periodic_constant_phase_shifter_analytic(N, phase_angle):
    n = np.arange(N)
    h = np.zeros(N)

    if N%2==0:
        n_odd = n[n%2==1]
        h[n%2==1] = 2 / N / np.tan(np.pi * n_odd / N)
    elif N%2==1:
        n_odd = n[n%2==1]
        n_even_nonzero = n[(n%2==0) & (n!=0)]
        h[n%2==1] = 1 / N / np.tan(np.pi * n_odd / 2 / N)
        h[(n%2==0) & (n!=0)] = 1 / N / np.tan(np.pi * (n_even_nonzero + N) / 2 / N)
    h[1:] *= -np.sin(phase_angle)
    h[0] = np.cos(phase_angle)
    return h

phase_angle = 0.25 * np.pi

N_even, N_odd = 32, 31
h_analytic_even = periodic_constant_phase_shifter_analytic(N_even, phase_angle)
h_analytic_odd = periodic_constant_phase_shifter_analytic(N_odd, phase_angle)

fig, Ax = plt.subplots(figsize=(12, 3), ncols=2, sharey=True)
Ax[0].stem(h_analytic_even)
Ax[1].stem(h_analytic_odd)
Ax[0].set_title('$N={:0.0f}$ (Analytic)'.format(N_even))
Ax[1].set_title('$N={:0.0f}$ (Analytic)'.format(N_odd))
for ax in Ax:
    ax.grid()
    ax.set_xlabel('$n$ / samples')

### DFT

Discrete and periodic signals can be represented in terms of
discrete Fourier series expansion which is equivalent to
the discrete Fourier transform (DFT).
Therefore, the periodic constant phase shifter can be
defined in terms of DFT spectrum,

\begin{align}
H[k] =
\begin{cases}
e^{+i\varphi},& k = 1,\ldots,\frac{M}{2}-1\\
e^{-i\varphi},& k = \frac{M}{2}+1, \ldots, M-1\\
\cos\varphi,& k = 0, \frac{M}{2}
\end{cases}
\end{align}

for even $M$, and

\begin{align}
H[k] =
\begin{cases}
e^{+i\varphi},& k = 1,\ldots,\frac{M - 1}{2}\\
e^{-i\varphi},& k = \frac{M + 1}{2}, \ldots, M-1\\
\cos\varphi,& k = 0
\end{cases}
\end{align}

for odd $M$.
The impulse response is the inverse DFT of $H[k]$.

In [ ]:
def periodic_constant_phase_shifter_idft(N, phase_angle):
    H = np.ones(N // 2 + 1, dtype='complex')
    H[0] = np.cos(phase_angle)
    if N % 2 is 0:
        H[1:-1] *= np.exp(1j * phase_angle)
        H[-1] = np.cos(phase_angle)
    elif N % 2 is 1:
        H[1:] *= np.exp(1j * phase_angle)
    return np.fft.irfft(H, n=N)

phase_angle = 0.25 * np.pi

N_even, N_odd = 32, 31
h_idft_even = periodic_constant_phase_shifter_idft(N_even, phase_angle)
h_idft_odd = periodic_constant_phase_shifter_idft(N_odd, phase_angle)

fig, Ax = plt.subplots(figsize=(12, 3), ncols=2, sharey=True)
Ax[0].stem(h_idft_even)
Ax[1].stem(h_idft_odd)
Ax[0].set_title('$N={:0.0f}$ (IDFT)'.format(N_even))
Ax[1].set_title('$N={:0.0f}$ (IDFT)'.format(N_odd))
for ax in Ax:
    ax.grid()
    ax.set_xlabel('$n$ / samples')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 3), ncols=2, sharey=True)
ax[0].plot(db(h_analytic_even - h_sum_even), '-o', label='sum')
ax[0].plot(db(h_analytic_even - h_idft_even), '-o', label='idft')
ax[1].plot(db(h_analytic_odd - h_sum_odd), '-o', label='sum')
ax[1].plot(db(h_analytic_odd - h_idft_odd), '-o', label='idft')
for axi in ax:
    axi.grid()
    axi.set_xlabel('$n$')
ax[0].set_title('$N={:0.0f}$'.format(N_even))
ax[1].set_title('$N={:0.0f}$'.format(N_odd))
ax[0].set_ylabel('Error / dB')
ax[0].legend();

## Example I - Castanets

In [ ]:
src_dir = '../data/source-signals'
filename = 'castanets'
suffix = '.wav'

s, fs = sf.read(join(src_dir, filename + suffix), start=5000, stop=12000)
n_sig = len(s)
t_sig = util.n2t(n_sig, fs, ms=True)

A phase shift can cause increased peak of the signal.
In order to have enough headroom,
the maximum amplitude of the source signal is normalized to the value `peak`.
The true peak `true_peak` is obtained from the oversampled signal.

In [ ]:
peak = -6
oversample = 4
true_peak = np.max(np.abs(resample(s, oversample * n_sig)))
s *= 10**(s / 20) / true_peak
db(np.max(np.abs(s)))

In [ ]:
repetition = 3
t_period, t_predelay, t_intro, t_outro = 500, 25, 15, 15  # in milliseconds
n_period, n_predelay, n_intro, n_outro = (np.array([t_period, t_predelay, t_intro, t_outro]) / 1000 * fs).astype('int')
n_fadein, n_fadeout = n_intro + n_predelay, n_outro + n_period - n_sig - n_predelay
n_fft = n_period
n_full = repetition * n_period + n_intro + n_outro

Properly chosen number of zeros are prepended and appended.

In [ ]:
s_oneperiod = np.zeros(n_period)
s_oneperiod[n_predelay + 1:n_predelay + 1 + n_sig] = s

The constant phase shift in the DFT domain is performed as follows.

- For input signal `x`, the DFT spectrum `X` is computed
- Since `x` is real, only the first half of the spectrum suffices
  (using `numpy.fft.rfft`)
- A constant phase shift is applied to the spectrum
  by multiplying the complex exponential `np.exp(1j * phi)`
  except the DC spectrum `X[0]` (and also `X[N-1]` if `N` is even).
- $\cos\varphi$ is multiplied to `X[0]` (and `X[N-1]` for even `N`).
- Finally, the phase shifted signal is obtained by
  inverse DFT `numpy.fft.irfft`.
  The DFT length has to be specified.

In [ ]:
def constant_phaseshift_dft(x, phi):
    N = len(x)
    X = np.fft.rfft(x)
    X[0] *= np.cos(phi)
    X[1:-1] *= np.exp(1j * phi)
    if N % 2 == 1:
        X[-1] *= np.exp(1j * phi)
    elif N % 2 == 0:
        X[-1] *= np.cos(phi)
    return np.fft.irfft(X, n=N)

In [ ]:
phase_angle = np.pi / 2

s_phaseshift = constant_phaseshift_dft(s_oneperiod, phase_angle)

In [ ]:
t = np.arange(n_period) * 1000 / fs

fig, ax = plt.subplots(figsize=(12, 4), ncols=2, gridspec_kw={'wspace':0.3})

ax[0].plot(t, s_oneperiod, label='Original')
ax[0].plot(t, s_phaseshift, label='Phase shifted')
ax[0].plot(t_predelay, 0, 'kx')
ax[0].plot(t_period - t_predelay, 0, 'kx')
ax[0].set_xlabel('$t$ / ms')
ax[0].grid()

ax[1].semilogx(db(np.fft.rfft(s_oneperiod)))
ax[1].semilogx(db(np.fft.rfft(s_phaseshift)))
ax[1].set_xlabel('$\mu$ (DFT bin)')
ax[1].set_ylabel('Magnitude / dB')
ax[1].set_title('DFT Spectrum')
ax[1].set_ylim(-100)
ax[1].grid()

The intro and outro positions are indicated by $\times$.

Although the phase shifted signal has a different peak,
the energy of the two signals are the same.

In [ ]:
db(np.linalg.norm(s_oneperiod)), db(np.linalg.norm(s_phaseshift))

The crest factors is different, though.

In [ ]:
db(util.crest_factor(s_oneperiod, oversample=4)), db(util.crest_factor(s_phaseshift, oversample=4))

The signal is now
* concatenated (using `np.tile`)
* prepended and appended by zeros (`n_intro` and `n_outro` respectively)
* fade-in and -out (`n_fadein` and `n_fadeout` respectively using `util.fade`)

In [ ]:
s_full = np.zeros(n_full)
s_full[n_intro:-n_outro] = np.tile(s_phaseshift, repetition)
s_full = util.fade(s_full, n_fadein, n_fadeout, type='h')

In [ ]:
t = np.arange(n_full) * 1000 / fs

fig, ax = plt.subplots(figsize=(12, 8), nrows=2, sharex=True)

ax[0].plot(t, s_full)
ax[1].plot(t, db(s_full))
ax[1].set_ylabel('Amplitude / dB')
ax[1].set_xlabel('$t$ / ms')
for axi in ax:
    axi.grid()
    for i in range(repetition + 1):
        axi.plot(t_intro + i * t_period, 0, 'kv')

The beginning and ending of each period are indicated
by black triangles.

## Example II - Square Wave Bursts

In [ ]:
fs = 44100
f0 = 50
num_partials = 10
amplitude = 0.25
duration = 10 / f0
phase_angle = 0.5 * np.pi
modal_window = kaiser(2 * num_partials + 1, beta=4)[num_partials + 1:]

_, square, _ = util.square_wave(f0, num_partials, amplitude, duration,
                                  fs, 0, modal_window)

n_sig = len(square)
t_sig = n_sig * 1000 / fs
t_sig, 1000 / f0

A square wave with phase shift 0 is first generated.

In [ ]:
t_taper = 2 / f0 * 1000
n_taper = int(t_taper / 1000 * fs)
n_taper

In [ ]:
square_tapered = util.fade(square, n_taper, n_taper, type='h')

t = np.arange(n_sig) * 1000 / fs

plt.plot(t, square, c='lightgray')
plt.plot(t, square_tapered)
plt.xlabel('$t$ / ms')
plt.grid();

In [ ]:
repetition = 3
t_period, t_predelay, t_intro, t_outro = 240, 20, 20, 20  # in milliseconds
n_period = util.t2n(t_period, fs=fs, ms=True)
n_predelay = util.t2n(t_predelay, fs=fs, ms=True)
n_intro = util.t2n(t_intro, fs=fs, ms=True)
n_outro = util.t2n(t_outro, fs=fs, ms=True)
n_fadein, n_fadeout = n_intro + n_predelay, n_outro + n_period - n_sig - n_predelay
n_fft = n_period
n_full = repetition * n_period + n_intro + n_outro

In [ ]:
square_oneperiod = np.zeros(n_period)
square_oneperiod[n_predelay + 1:n_predelay + 1 + n_sig] = square_tapered
t = np.arange(n_period) * 1000 / fs

square_phaseshift = util.constant_phase_shift_dft(square_oneperiod, -np.pi / 2)

fig, ax = plt.subplots(figsize=(12, 4), ncols=2, gridspec_kw={'wspace':0.3})

ax[0].plot(t, square_oneperiod)
ax[0].plot(t, square_phaseshift)
ax[0].set_xlabel('$t$ / ms')
ax[0].grid();

ax[1].semilogx(db(np.fft.rfft(square_oneperiod)))
ax[1].semilogx(db(np.fft.rfft(square_phaseshift)))
ax[1].set_xlabel('$\mu$ (DFT bin)')
ax[1].set_ylabel('Magnitude / dB')
ax[1].set_title('DFT Spectrum')
ax[1].set_ylim(-100)
ax[1].grid()

In [ ]:
square_oneperiod.shape, square_phaseshift.shape, n_full - n_intro - n_outro

A constant phase shift is applied in the DFT domain
by using the utility function `util.constant_phase_shift_dft`.

The crest factor of a square wave is very
sensitive to phase shift.

In [ ]:
db(util.crest_factor(square_oneperiod, oversample=4)), db(util.crest_factor(square_phaseshift, oversample=4))